In [1]:
!pwd

#import pandas
from pprint import pprint # we use this to pretty print some stuff later
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Users/kpadhikari/Desktop/BigFls/CLAS12/GitProj/KPAdhikari/PythonStuff/IPython_Jupyter_Notebooks/ML_Stuff


## Hands-On Tutorial On Machine Learning Pipelines With Scikit-Learn
15/10/2020

Ref: https://analyticsindiamag.com/hands-on-tutorial-on-machine-learning-pipelines-with-scikit-learn/

With increasing demand in [machine learning](https://analyticsindiamag.com/top-8-books-on-machine-learning-in-cybersecurity-one-must-read/) and [data science in businesses](https://analyticsindiamag.com/23-latest-data-science-jobs-from-tech-giants-like-amazon-google-more/), for upgraded data strategizing there’s a need for a better workflow to ensure robustness in [data modelling](https://analyticsindiamag.com/50-latest-data-science-and-analytics-jobs-that-opened-last-week/). **Machine learning has certain steps** to be followed namely – **data collection, data preprocessing (cleaning and [feature engineering](https://analyticsindiamag.com/image-feature-extraction-using-scikit-image-a-hands-on-guide/)), model training, validation and prediction** on the test data (which is previously unseen by model). 

Here testing data needs to go through the same preprocessing as training data. For this iterative process, pipelines are used which can automate the entire process for both training and testing data. It ensures reusability of the model by reducing the redundant part, thereby speeding up the process. This could prove to be very effective during the [**production workflow**](https://analyticsindiamag.com/machine-learning-research-management-dan-malowany/).

In this article, I’ll be discussing how to implement a machine learning pipeline using scikit-learn.

### Advantages of using Pipeline:
* Automating the workflow being iterative.
* Easier to fix bugs 
* Production Ready
* Clean code writing standards
* Helpful in iterative hyperparameter tuning and cross-validation evaluation

### Challenges in using Pipeline:
* Proper data cleaning
* Data Exploration and Analysis
* Efficient feature engineering

### Scikit-Learn Pipeline
The sklearn.pipeline module implements utilities to build a composite estimator, as a chain of transforms and estimators.

I’ve used the Iris dataset which is readily available in scikit-learn’s datasets library. The 6 columns in this dataset are: Id, SepalLength(in cm), SepalWidth(in cm), PetalLength(in cm), PetalWidth(in cm), Species(Target). 50samples containing 3 classes-Iris setosa, Iris Virginica, Iris versicolor.

After loading the data, split it into training and testing then build pipeline object wherein standardization is done using StandardScalar() and dimensionality reduction using PCA(principal component analysis) both of these with be fit and transformed(these are transformers), lastly the model to use is declared here it is LogisticRegression, this is the estimator. The pipeline is fitted and the model performance score is determined.

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
iris_df=load_iris()
X_train,X_test,y_train,y_test=train_test_split(iris_df.data,iris_df.target,test_size=0.3,random_state=0)
pipeline_lr=Pipeline([('scalar1',StandardScaler()),
                     ('pca1',PCA(n_components=2)),                     ('lr_classifier',LogisticRegression(random_state=0))])
model = pipeline_lr.fit(X_train, y_train)
model.score(X_test,y_test)

0.8666666666666667

    OUTPUT - 0.8666666666666667

With the pipeline, we preprocess the training data and fit the model in a single line of code. In contrast, without a pipeline, we have to do normalization, dimensionality reduction, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables.

Use the following two lines of code inside the Pipeline object for filling missing values and change categorical values to numeric. (Since iris dataset doesn’t contain these we are not using)

```py
('imputer', SimpleImputer(strategy='most_frequent')) #filling missing values

(‘onehot', OneHotEncoder(handle_unknown='ignore'))    #convert categorical 
```

Make sure to import OneHotEncoder and SimpleImputer modules from sklearn!

### Stacking Multiple Pipelines to Find the Model with the Best Accuracy

We build different pipelines for each algorithm and the fit to see which performs better.

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
pipeline_lr=Pipeline([('scalar1',StandardScaler()),
                     ('pca1',PCA(n_components=2)), 
                     ('lr_classifier',LogisticRegression())])
pipeline_dt=Pipeline([('scalar2',StandardScaler()),
                     ('pca2',PCA(n_components=2)),
                     ('dt_classifier',DecisionTreeClassifier())])
pipeline_svm = Pipeline([('scalar3', StandardScaler()),
                      ('pca3', PCA(n_components=2)),
                      ('clf', svm.SVC())])
pipeline_knn=Pipeline([('scalar4',StandardScaler()),
                     ('pca4',PCA(n_components=2)),
                     ('knn_classifier',KNeighborsClassifier())])
#pipelines = [pipeline_lr, pipeline_dt, pipeline_randomforest, pipeline_knn]
pipelines = [pipeline_lr, pipeline_dt, pipeline_svm, pipeline_knn]
pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Support Vector Machine',3:'K Nearest Neighbor'}
for pipe in pipelines:
  pipe.fit(X_train, y_train)
for i,model in enumerate(pipelines):
    print("{} Test Accuracy:{}".format(pipe_dict[i],model.score(X_test,y_test)))

Logistic Regression Test Accuracy:0.8666666666666667
Decision Tree Test Accuracy:0.9111111111111111
Support Vector Machine Test Accuracy:0.9333333333333333
K Nearest Neighbor Test Accuracy:0.9111111111111111


    OUTPUT:
    Logistic Regression Test Accuracy: 0.8666666666666667
    Decision Tree Test Accuracy: 0.9111111111111111
    Support Vector Machine Test Accuracy: 0.9333333333333333
    K Nearest Neighbor Test Accuracy: 0.9111111111111111
    From the results, it’s clear that Support Vector Machines(SVM) perform better than other models.

### Hyperparameter Tuning in Pipeline
With pipelines, you can easily perform a **grid-search over a set of parameters** for each step of this meta-estimator to find the best performing parameters. To do this you first need to create a **parameter grid** for your chosen model. One important thing to note is that you need to append the name that you have given the classifier part of your pipeline to each parameter name. In my code above I have called this ‘randomforestclassifier’ so I have added randomforestclassifier__ to each parameter. Next, I created a grid search object which includes the original pipeline. When I then call fit, the transformations are applied to the data, before a cross-validated grid-search is performed over the parameter grid.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = make_pipeline((RandomForestClassifier()))
grid_param = [
{"randomforestclassifier": [RandomForestClassifier()],
"randomforestclassifier__n_estimators":[10,100,1000],                    
 "randomforestclassifier__max_depth":[5,8,15,25,30,None],                 
 "randomforestclassifier__min_samples_leaf":[1,2,5,10,15,100],
"randomforestclassifier__max_leaf_nodes": [2, 5,10]}]
gridsearch = GridSearchCV(pipe, grid_param, cv=5, verbose=0,n_jobs=-1) 
best_model = gridsearch.fit(X_train,y_train)
best_model.score(X_test,y_test)
print(best_model.score(X_test,y_test))

    OUTPUT - 0.9777777777777777

### Conclusion
This is a basic pipeline implementation. In real-life data science, scenario data would need to be prepared first then applied pipeline for rest processes. Building quick and efficient machine learning models is what pipelines are for. Pipelines are high in demand as it helps in coding better and extensible in implementing big data projects. Automating the applied machine learning workflow and saving time invested in redundant preprocessing work.

The complete code of the above implementation is available at the AIM’s GitHub repository. Please visit [this link](https://github.com/analyticsindiamagazine/AIM-Code-Repo/blob/main/Developers%20Corner/ML%20pipeline.ipynb) to find the notebook with codes.

# Pickle in Python
Ref: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0720EN-SkillsNetwork/labs/Module 1/pickle.md.html

Time Effort: 30 mins

#### Learning Objective :

    What is Pickling?

    Why we need Pickling?

    What can pickle be used for?

    When not to use pickle

    What can be pickled?

    Pickle in the peer graded assignment

#### What is Pickling?
When working with dictionaries, DataFrames, or any other data type, you might want to save them to a file, so you can use them later on or send them to someone else. This is what Python pickle module is for: it serializes objects so they can be saved to a file, and loaded in a program again later on.

Pickle is used for **serializing and de-serializing** Python object structures, also called marshalling or flattening. Serialization refers to the process of converting an object in memory to a byte stream that can be stored on disk or sent over a network. Later on, this character stream can then be retrieved and de-serialized back to a Python object. The conversion of an object from one representation (data in Random Access Memory (RAM)) to another (text on disk), while the latter is the process of encoding data with fewer bits, in order to save disk space.

#### Why we need Pickling?
Storing the state of an object in a file or database can save time to process huge datasets in many data science projects. For example, you only need to pre-process the dataset once and save the model into a disk. Later you just need to deserialize it and reuse the pre-cooked model as many times as you want. This is definitely preferred to pre-processing it each time.

#### What can pickle be used for?
Pickling is useful for applications where you need some degree of persistency in your data. Your programs state data can be saved to disk, so you can continue working on it later on. Pickle is very useful for when you're working with machine learning algorithms, where you want to save them to be able to make new predictions at a later time, without having to rewrite everything or train the model all over again.

#### When not to use pickle
If you want to use data across different programming languages, pickle is not recommended. Its protocol is specific to Python, thus, cross-language compatibility is not guaranteed. The same holds for different versions of Python itself. Unpickling a file that was pickled in a different version of Python may not always work properly, so you have to make sure that you're using the same version and perform an update if necessary.

#### What can be pickled?
You can pickle objects with the following data types: Booleans, Integers, Floats, Complex numbers, (normal and Unicode) Strings, Tuples, Lists, Sets, and Dictionaries that ontain picklable objects. All the data types be pickled, but you can also do the same for classes and functions, for example, if they are defined at the top level of a module.

#### Pickle in the peer graded assignment
In the case of the capstone project, the 2 datasets are already present but they are very huge and reading them through the traditional approach each time is time consuming.

So first time read it . Later convert it into byte stream (serialize) and the file which is in the serialized form is the pickle file.

Here you are serializing the dataframe object(converting to bytes ) and saving it using a pickle file in the steps
```py
df.to_pickle('./df_raw.pkl')
```
Later uploading it to cloud storage using the command.
```py
client_cred.upload_file('./df_raw.pkl',bucket,'df_raw_cos.pkl')
```
To upload the file we need the client_credentials.

This is present in the autogenerated code.

For convenience in naming variables assign the autogenerated client credential variable to a new variable client_cred.
```py
client_cred= client_32cef856d9ba404c8f3df1dfc0c8cd51
```
**Note: The autogenerated client credentials variable is usually in this format.It will have the word prefix as client followed by alphanumeric characters. For Example : client32cef856d9ba404c8f3df1dfc0c8cd51) The autogenerated variable name may vary in the notebook.Assign the one generated in your notebook.**

Next we want to get back the dataframe from its saved location. For that we need to do the following steps.

Download the file from Cloud Object Store:

    client_cred.download_file(Bucket=bucket,Key='df_raw_cos.pkl',Filename='./df_raw_local.pkl') 
    
Hence we need the bucket variable value.

Here you are downloading the uploaded pickle file using your credentials information,converting the bytestream back to dataframe and reading the dataframe.

So initially we create a bucket variable and assign it to. Next we replace the *** string with the bucket string that you got when you created a Panda Dataframe from the uploaded file in the previous section.

The bucket string value is got from the Bucket key value present in the autogenerated code:

    clientcredentials.get_object(Bucket=" ")
Author(s)

Malika Singla

Other Contributor(s)
Lakshmi Holla

## Tmp: Some work as part of EdX course "Data Science and Machine Learning Capstone Project"


In [5]:
#import pandas
from pprint import pprint # we use this to pretty print some stuff later
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
import pandas as pd
def makeSavePickles(file):
    df = pd.read_csv('{}.csv'.format(file))    
    df.to_pickle('{}.pkl'.format(file))

In [10]:
path = "/Users/kpadhikari/Desktop/BigFls/DS_ML"

#### Here I want to make pkl files of all 5 PLUTO data sets (corresponding to 5 NYC boroughs)
####   Later I will disable these lines (pkl files have smaller size than corresponding .csv
####    and also it saves time to load them up later on into a dataframe.)
#### I was wrong to say that pkl files are always smaller in size than the corresponding
####    csv/ASCII files (I said that based on my observation of the 311 dataset. In fact all of
####    the PLUTO data file got bigger when converted to pkl (see below))
#makeSavePickles('{}/PLUTO_for_WEB/BK_18v1'.format(path)) #Brooklyn: 82.5 MB to 193.3 MB
#makeSavePickles('{}/PLUTO_for_WEB/BX_18v1'.format(path)) #Bronx:  26.9 MB to 62.4 MB
#makeSavePickles('{}/PLUTO_for_WEB/MN_18v1'.format(path)) #Manhattan: 13.6 to 29.8 MB
#makeSavePickles('{}/PLUTO_for_WEB/QN_18v1'.format(path)) #Queens:    96.6 to 225.2 MB
#makeSavePickles('{}/PLUTO_for_WEB/SI_18v1'.format(path)) #Staten Island: 37.2 to 86 MB


#This is a big file (2.4G)
file = "311_Service_Requests_from_2010_to_Present_min" #older (differently formatted?)

#excel_data_df = pandas.read_excel('records.xlsx', sheet_name='Employees')
#df = pandas.read_excel('tableRBEs_orgAndOurs.xlsx') #, sheet_name='Employees')
#df0 = pd.read_excel('{}/{}'.format(path,file)) #, sheet_name='Employees')
#df0 = pd.read_csv('{}/{}.csv'.format(path,file)) #, sheet_name='Employees')
#df0.head()
#df0 = None
#df0.head() #AttributeError: 'NoneType' object has no attribute 'head'

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (19,20,22,23,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (20,23,24,26,64) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (19,20,22,23,64,65,77) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (19,20,22,23,24,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=as

In [7]:
#####display(df0.columns)
#display(df0.info)
#####display(df0.dtypes)

In [8]:
#https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0720EN-SkillsNetwork/labs/Module%201/pickle.md.html
# Following line created a .pkl file of size 658.9 MB, while the original csv file had a size of 2.55 GB 
#      (which is about 75% reduction in size)
###### df0.to_pickle('/Users/kpadhikari/Desktop/BigFls/DS_ML/311_Service_Requests_from_2010_to_Present_min.pkl')
## 
## Now reading the pickle file that was first created out of the dataframe that was made out of .csv data
dfpk0 = pd.read_pickle('{}/{}.pkl'.format(path,file))

In [9]:
display(dfpk0.columns)
#display(df0.info)
display(dfpk0.dtypes)
dfpk0.head()

Index(['Unnamed: 0', 'Unique Key', 'Created Date', 'Closed Date',
       'Complaint Type', 'Location Type', 'Incident Zip', 'Incident Address',
       'Street Name', 'Address Type', 'City', 'Status',
       'Resolution Description', 'Borough', 'Latitude', 'Longitude'],
      dtype='object')

Unnamed: 0                  int64
Unique Key                  int64
Created Date               object
Closed Date                object
Complaint Type             object
Location Type              object
Incident Zip              float64
Incident Address           object
Street Name                object
Address Type               object
City                       object
Status                     object
Resolution Description     object
Borough                    object
Latitude                  float64
Longitude                 float64
dtype: object

,Unnamed: 0,Unique Key,Created Date,Closed Date,Complaint Type,Location Type,Incident Zip,Incident Address,Street Name,Address Type,City,Status,Resolution Description,Borough,Latitude,Longitude
0,0,45531130,02/02/2020 06:09:17 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10019.0,426 WEST 52 STREET,WEST 52 STREET,ADDRESS,NEW YORK,Open,The following complaint conditions are still o...,MANHATTAN,40.765132,-73.988993
1,1,45529784,02/02/2020 02:15:24 PM,NaN,UNSANITARY CONDITION,RESIDENTIAL BUILDING,11204.0,1751 67 STREET,67 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.618484,-73.992673
2,2,45527528,02/02/2020 02:27:41 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,11372.0,87-15 37 AVENUE,37 AVENUE,ADDRESS,Jackson Heights,Open,The following complaint conditions are still o...,QUEENS,40.750269,-73.879432
3,3,45530329,02/02/2020 12:13:18 PM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10458.0,2405 SOUTHERN BOULEVARD,SOUTHERN BOULEVARD,ADDRESS,BRONX,Open,The following complaint conditions are still o...,BRONX,40.853773,-73.881558
4,4,45528814,02/02/2020 01:59:44 PM,NaN,APPLIANCE,RESIDENTIAL BUILDING,11209.0,223 78 STREET,78 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.629745,-74.030533


#### Module 1: Quiz
* Q1: The dataset contains complaints logged since what date?
* Q2: The dataset contains complaints logged till what date?
* Q3: How many incidents have a missing Incident Address?
* Q4: How many valid ZIP Codes exist in the Bronx PLUTO dataset?
* Q5: How many valid ZIP Codes exist in the Queens PLUTO dataset?

References for some of the ideas below:
* https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e
* https://stackoverflow.com/questions/57859504/pandas-transform-date-string-from-format-mm-d-yyyy-and-mm-dd-yyyy-to-dd-mm-yyyy


In [10]:
import pandas as pd

# example data frame with dates in the format mm/d/yyyy and mm/dd/yyyy
dfdtEx = pd.DataFrame({'date' : ['12/1/2008','12/5/2008','12/10/2008','12/17/2008']})
display(dfdtEx.head())
print(dfdtEx['date'].isna().sum()) #Counting Empty or Not-available or Missing values in the 'date' column

,date
0,12/1/2008
1,12/5/2008
2,12/10/2008
3,12/17/2008


0


In [11]:
dfdtEx['date'] = pd.to_datetime(dfdtEx['date'])
dfdtEx['date'] = dfdtEx['date'].dt.strftime('%m.%d.%Y')
dfdtEx.head()

,date
0,12.01.2008
1,12.05.2008
2,12.10.2008
3,12.17.2008


If you want to extract days, months, years or so, pandas has a special dt functionality for datetime types, hence, you need to convert your column first into that type.

You can access days and months like this:

In [12]:
dfdtEx['date'] = pd.to_datetime(dfdtEx['date'])
dfdtEx['month'] = dfdtEx['date'].dt.month
dfdtEx['day'] = dfdtEx['date'].dt.day
dfdtEx['year'] = dfdtEx['date'].dt.year
dfdtEx.head()

,date,month,day,year
0,2008-12-01,12,1,2008
1,2008-12-05,12,5,2008
2,2008-12-10,12,10,2008
3,2008-12-17,12,17,2008


In [13]:
display(min(dfdtEx['date']))
display(max(dfdtEx['date']))

Timestamp('2008-12-01 00:00:00')

Timestamp('2008-12-17 00:00:00')

In [14]:
display(min(dfpk0['Created Date']))
display(max(dfpk0['Created Date']))

'01/01/2010 02:31:13 PM'

'12/31/2019 12:59:12 PM'

dfpk0.head() above showed dates from 2020 but the max method showed it to be 2019, so I suspect, I have to first parse the above date column to **datetime** type.

In [15]:
#https://stackoverflow.com/questions/51235708/parsing-string-to-datetime-while-accounting-for-am-pm-in-pandas
#https://www.dataindependent.com/pandas/pandas-to-datetime/ 
print(pd.to_datetime("2018 - 07 - 07 04 - PM", format='%Y - %m - %d %I - %p'))

2018-07-07 16:00:00


In [16]:
#https://www.dataindependent.com/pandas/pandas-to-datetime/
#dfpk0['Created Date'] = pd.to_datetime(dfpk0['Created Date']) #didn't work
dfpk0['Created Date'] = pd.to_datetime(dfpk0['Created Date'], format='%m/%d/%Y %H:%M:%S %p') #kp: Removes AM or PM
#dfpk0.head()

In [17]:
display(min(dfpk0['Created Date']))
display(max(dfpk0['Created Date']))

Timestamp('2010-01-01 02:31:13')

Timestamp('2020-02-02 12:55:36')

In [18]:
print(dfpk0['Incident Address'].isna().sum()) #Counting Empty or Not-available or Missing values in the column

52825


In [19]:
dfbx = pd.read_csv('{}/PLUTO_for_WEB/BX_18v1.csv'.format(path))
display(dfbx.head())

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (19,20,22,23,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version
0,BX,2260,1,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
1,BX,2260,4,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
2,BX,2260,10,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
3,BX,2260,17,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
4,BX,2260,18,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1


In [20]:
dfqn = pd.read_csv('{}/PLUTO_for_WEB/QN_18v1.csv'.format(path))
display(dfqn.head())

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (19,20,22,23,64,65,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version
0,QN,6,1,402,1.0,NaN,30.0,26.0,11101.0,L115,...,Y,401 011,40101.0,NaN,4.000060e+09,09/20/2013,1,1.0,1.0,18V1
1,QN,6,3,402,1.0,1015.0,30.0,26.0,11101.0,L115,...,NaN,401 011,40101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1
2,QN,6,8,402,1.0,1011.0,30.0,26.0,11101.0,L115,...,NaN,401 011,40101.0,NaN,4.000060e+09,08/07/2013,1,1.0,1.0,18V1
3,QN,6,20,402,1.0,NaN,30.0,26.0,11101.0,L115,...,NaN,401 011,40101.0,NaN,4.000060e+09,09/20/2013,1,1.0,1.0,18V1
4,QN,6,30,402,1.0,NaN,30.0,26.0,11101.0,L115,...,NaN,401 011,40101.0,NaN,4.000060e+09,09/08/2017,1,1.0,1.0,18V1


In [21]:
#Grouping and perform count over each group
dfzc =  dfbx.groupby('ZipCode')['ZipCode'].count()
display(dfzc)
display(dfzc.count())

ZipCode
10451.0     1228
10452.0     1548
10453.0     2102
10454.0     1746
10455.0     1793
10456.0     2876
10457.0     3028
10458.0     2769
10459.0     2605
10460.0     2892
10461.0     7499
10462.0     4671
10463.0     2247
10464.0     1550
10465.0     8163
10466.0     9229
10467.0     4673
10468.0     1627
10469.0    11349
10470.0     2334
10471.0     2352
10472.0     4329
10473.0     4870
10474.0     1220
10475.0      823
11370.0        2
Name: ZipCode, dtype: int64

26

In [22]:
#dfbx.groupby('ZipCode')['ZipCode'].sum()
dfzc =  dfqn.groupby('ZipCode')['ZipCode'].count()
display(dfzc)
display(dfzc.count())

ZipCode
11001.0    1241
11004.0    2502
11005.0       1
11040.0     589
11101.0    3263
           ... 
11692.0    2384
11693.0    1963
11694.0    3743
11695.0       6
11697.0      13
Name: ZipCode, Length: 65, dtype: int64

65

## Module 2: What Is the Top Complaint Type?
Estimated time needed: 2 hours

### Objective for Exercise:
Use data science methodologies to define and formulate a real-world business problem.
The goal of this exercise is to find the answer to the Question 1 of the problem statement:

**Which type of complaint should the Department of Housing Preservation and Development of New York City focus on first?**

In this exercise, you need to read back the 311 datasets that you stored in Cloud Object Store and explore the dataset.

By the end of this exercise, you need to figure out the correct Complaint Type that the Department of Housing Preservation and Development of New York City should focus on.


### Module 2: Quiz
* What is the total number of complaints that exist in the dataset?
* How many different Complaint Types can you find in the dataset, including duplicates entries of the same type?
* How many Elevator complaints can you find in the dataset?
* How many Electric complaints can you find in the dataset?
* Using 800,000 as a threshold, what complaint type(s) do you recommend the Department of Housing Preservation and Development of New York City address first? Select all that apply.

In [23]:
display(dfpk0.columns)
#display(df0.info)
display(dfpk0.dtypes)
display(dfpk0.count())

Index(['Unnamed: 0', 'Unique Key', 'Created Date', 'Closed Date',
       'Complaint Type', 'Location Type', 'Incident Zip', 'Incident Address',
       'Street Name', 'Address Type', 'City', 'Status',
       'Resolution Description', 'Borough', 'Latitude', 'Longitude'],
      dtype='object')

Unnamed: 0                         int64
Unique Key                         int64
Created Date              datetime64[ns]
Closed Date                       object
Complaint Type                    object
Location Type                     object
Incident Zip                     float64
Incident Address                  object
Street Name                       object
Address Type                      object
City                              object
Status                            object
Resolution Description            object
Borough                           object
Latitude                         float64
Longitude                        float64
dtype: object

Unnamed: 0                6019843
Unique Key                6019843
Created Date              6019843
Closed Date               5893186
Complaint Type            6019843
Location Type             5967019
Incident Zip              5939146
Incident Address          5967018
Street Name               5967018
Address Type              5935078
City                      5939569
Status                    6019843
Resolution Description    6012017
Borough                   6019843
Latitude                  5939172
Longitude                 5939172
dtype: int64

So, the answer to "**What is the total number of complaints that exist in the dataset?**" is **6019843** because that's the maximum # of rows (the lower values of count() for other columns comes because of the fact that they have some rows with missing values).

In [24]:
dfzc =  dfpk0.groupby('Complaint Type')['Complaint Type'].count()
display(dfzc)
display(dfzc.count())

Complaint Type
AGENCY                          9
APPLIANCE                  112831
Appliance                       4
CONSTRUCTION                 5078
DOOR/WINDOW                205278
ELECTRIC                   307310
ELEVATOR                     6725
Electric                        1
FLOORING/STAIRS            137402
GENERAL                    151308
GENERAL CONSTRUCTION       500863
General                      1163
HEAT/HOT WATER            1261574
HEATING                    887850
HPD Literature Request      52824
Mold                            1
NONCONST                   260890
OUTSIDE BUILDING             7142
Outside Building                6
PAINT - PLASTER            361257
PAINT/PLASTER              346438
PLUMBING                   711130
Plumbing                       11
SAFETY                      51529
STRUCTURAL                     16
Safety                        424
UNSANITARY CONDITION       451643
Unsanitary Condition         5499
VACANT APARTMENT                6

30

So, the answer to **How many different Complaint Types can you find in the dataset, including duplicates entries of the same type?** is **30**

So, the answer to **How many Elevator complaints can you find in the dataset?** is **6725**

So, the answer to **How many Electric complaints can you find in the dataset?** is **307311**, which is a total of 307310 for ELECTRIC type and 1 for 'electric' type, which are the same thing but perhaps due to some memory issues by the data recorder, the new complaint type 'electric' was created again.

So, the answer to **Using 800,000 as a threshold, what complaint type(s) do you recommend the Department of Housing Preservation and Development of New York City address first? Select all that apply.** is **(B) Heating** and **(E)Both of 'Heat/Hot Water'**.

## Module 3: What Areas Should the Agency Focus On?
Estimated time needed: 2 hours

### Objective for Exercise:
Use your data analysis tools to ingest a dataset, clean it, and wrangle it.
The goal of this exercise is to do explore the data to find the answer to the Question 2 problem statement:

Should the Department of Housing Preservation and Development of New York City focus on any particular set of boroughs, ZIP codes, or street (where the complaints are severe) for the specific type of complaints you identified in response to Question 1?

In this exercise, you will use 311 Dataset to determine whether to focus on any particular borough, ZIP code, or street (where the complaints are severe) for the specific Complaint Type you decided to focus at the end of the last exercise.



### Module 3: Quiz on Affected Areas
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, which borough had the highest number of complaints submitted?
* For the complaint types that you selected in the previous module that had a total number that exceeded 800,000 complaints, which borough had the lowest number of complaints submitted?
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, which ZIP code had the highest number of complainted submitted?
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, the address 89-21 Elmhurst Avenue had the highest number of complainted submitted?
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, how many of the submitted tickets were closed?

### (5) IF condition with OR
Ref: https://datatofish.com/if-condition-in-pandas-dataframe/

In the final case, let’s apply these conditions:

* If the name is ‘Bill’ or ‘Emma,’ then assign the value of ‘Match’
* Otherwise, if the name is neither ‘Bill’ nor ‘Emma,’ then assign the value of ‘Mismatch’


In [25]:
#https://datatofish.com/if-condition-in-pandas-dataframe/
#dfct['name_match'] = dfpk0['First_name'].apply(lambda x: 'Match' if x == 'Bill' else 'Mismatch')
#(5) IF condition with OR
import pandas as pd

names = {'First_name': ['Jon','Bill','Maria','Emma']}
df = pd.DataFrame(names,columns=['First_name'])
display(df)
df.loc[(df['First_name'] == 'Bill') | (df['First_name'] == 'Emma'), 'name_match'] = 'Match'  
df.loc[(df['First_name'] != 'Bill') & (df['First_name'] != 'Emma'), 'name_match'] = 'Mismatch'  

print (df)


,First_name
0,Jon
1,Bill
2,Maria
3,Emma


  First_name name_match
0        Jon   Mismatch
1       Bill      Match
2      Maria   Mismatch
3       Emma      Match


In [26]:
dfzc =  dfpk0.groupby('Complaint Type')['Complaint Type'].count()
display(dfzc.head())
display(dfzc.count())
#display(dfzc)
display(dfzc.shape)

Complaint Type
AGENCY               9
APPLIANCE       112831
Appliance            4
CONSTRUCTION      5078
DOOR/WINDOW     205278
Name: Complaint Type, dtype: int64

30

(30,)

In [53]:
dfgg = dfpk0.groupby(['Complaint Type', 'Borough']) 
# Print the first value in each group 
#dfgg.first() 

In [30]:
#dfgg.count()

Unnamed: 0  Unique Key  Created Date  \
Complaint Type Borough                                               
AGENCY         BRONX                   3           3             3   
               BROOKLYN                1           1             1   
               MANHATTAN               3           3             3   
               QUEENS                  2           2             2   
APPLIANCE      BRONX               33031       33031         33031   
...                                  ...         ...           ...   
WATER LEAK     BRONX               62374       62374         62374   
               BROOKLYN            69613       69613         69613   
               MANHATTAN           36739       36739         36739   
               QUEENS              21417       21417         21417   
               STATEN ISLAND        3488        3488          3488   

                              Closed Date  Location Type  Incident Zip  \
Complaint Type Borough                                                   
AGENCY         BRONX                    3              3             3   
               BROOKLYN                 1              1             1   
               MANHATTAN                3              3             2   
               QUEENS                   2              2             2   
APPLIANCE      BRONX                32018          33031         33009   
...                                   ...            ...           ...   
WATER LEAK     BRONX                61827          62374         62330   
               BROOKLYN             69348          69613         69572   
               MANHATTAN            36332          36739         36699   
               QUEENS               21303          21417         21379   
               STATEN ISLAND         3479           3488          3488   

                              Incident Address  Street Name  Address Type  \
Complaint Type Borough                                                      
AGENCY         BRONX                         3            3             3   
               BROOKLYN                      1            1             1   
               MANHATTAN                     3            3             2   
               QUEENS                        2            2             2   
APPLIANCE      BRONX                     33031        33031         33012   
...                                        ...          ...           ...   
WATER LEAK     BRONX                     62374        62374         62352   
               BROOKLYN                  69613        69613         69575   
               MANHATTAN                 36739        36739         36714   
               QUEENS                    21417        21417         21411   
               STATEN ISLAND              3488         3488          3488   

                               City  Status  Resolution Description  Latitude  \
Complaint Type Borough                                                          
AGENCY         BRONX              3       3                       3         3   
               BROOKLYN           1       1                       1         1   
               MANHATTAN          2       3                       3         2   
               QUEENS             2       2                       2         2   
APPLIANCE      BRONX          33009   33031                   33029     33009   
...                             ...     ...                     ...       ...   
WATER LEAK     BRONX          62330   62374                   62372     62330   
               BROOKLYN       69574   69613                   69611     69572   
               MANHATTAN      36699   36739                   36738     36699   
               QUEENS         21391   21417                   21416     21379   
               STATEN ISLAND   3488    3488                    3488      3488   

                              Longitude  
Complaint Type Borough                   
AGENCY         B

In [40]:
print(type(dfgg))
#dfgg.count().loc[(dfgg['Complaint Type'] == 'HEATING') | (dfgg['Complaint Type'] == 'HEAT/HOT WATER')]
#dfgg.get_group('HEATING')

#for name_of_the_group, group in dfgg:
#    print (name_of_the_group)
#    print (group)


<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [44]:
#dfgg.count().describe()

In [56]:
#https://stackoverflow.com/questions/42240476/python-pandas-groupby-filter-according-to-condition-on-values
#dfgg = dfpk0.groupby(['Complaint Type', 'Borough']) 
#df1['conditions_apply'] = (df1.foo >= lower_bound) & (df1.foo <= upper_bound)
#selection = df1.groupby('bar')['conditions_apply'].min()  # any False will return False
#selection = selection[selection].index.tolist()           # get all bars with Trues
#df1 = df1[df1.bar.isin(selection)]                        # make selection
#df1.drop(columns=['conditions_apply'], inplace=True)      # drop newly made column

dfpk0['conditions_apply'] = (dfpk0['Complaint Type'] == 'HEATING') & (dfpk0['Complaint Type'] == 'HEAT/HOT WATER')
selection1 = dfpk0.groupby('Borough')['conditions_apply'].count()
print(selection1)
#https://stackoverflow.com/questions/10202570/find-row-where-values-for-column-is-maximal-in-a-pandas-dataframe
print(selection1[selection1==selection1.max()])
selection2 = dfpk0.groupby('Incident Zip')['conditions_apply'].count()
display(selection2)
print(selection2[selection2==selection2.max()])#print(selection2.max())
selection3 = dfpk0.groupby('Incident Address')['conditions_apply'].count()
#print(selection3) #Too many results????
print(selection3[selection3==selection3.max()])#print(selection3.max())
selection4 = dfpk0.groupby('Status')['conditions_apply'].count()
print(selection4) #Too many results????
print(selection4[selection4==selection4.max()])#print(selection3.max())
dfpk0.drop(columns=['conditions_apply'], inplace=True)      # drop newly made column

Borough
BRONX            1617956
BROOKLYN         1739886
MANHATTAN        1055225
QUEENS            645971
STATEN ISLAND      87584
Unspecified       873221
Name: conditions_apply, dtype: int64
Borough
BROOKLYN    1739886
Name: conditions_apply, dtype: int64


Incident Zip
10001.0     9031
10002.0    32385
10003.0    25574
10004.0      329
10005.0      440
           ...  
11692.0    12498
11693.0     4768
11694.0    10557
11697.0      269
12345.0        1
Name: conditions_apply, Length: 202, dtype: int64

Incident Zip
11226.0    215709
Name: conditions_apply, dtype: int64
Incident Address
34 ARDEN STREET    14298
Name: conditions_apply, dtype: int64
Status
Assigned             4
Closed         5886253
In Progress        364
Open            133220
Pending              2
Name: conditions_apply, dtype: int64
Status
Closed    5886253
Name: conditions_apply, dtype: int64


In [55]:
#dfpk0['conditions_apply'] = (dfpk0['Complaint Type'] == 'HEATING') #| (dfpk0['Complaint Type'] == 'HEAT/HOT WATER')
dfpk0['conditions_apply'] = (dfpk0['Complaint Type'] == 'HEAT/HOT WATER')
selection1 = dfpk0.groupby('Borough')['conditions_apply'].count()
print(selection1)
#https://stackoverflow.com/questions/10202570/find-row-where-values-for-column-is-maximal-in-a-pandas-dataframe
print(selection1[selection1==selection1.max()])
selection2 = dfpk0.groupby('Incident Zip')['conditions_apply'].count()
display(selection2)
print(selection2[selection2==selection2.max()])#print(selection2.max())
selection3 = dfpk0.groupby('Incident Address')['conditions_apply'].count()
#print(selection3) #Too many results????
print(selection3[selection3==selection3.max()])#print(selection3.max())
selection4 = dfpk0.groupby('Status')['conditions_apply'].count()
print(selection4) #Too many results????
print(selection4[selection4==selection4.max()])#print(selection3.max())
dfpk0.drop(columns=['conditions_apply'], inplace=True)      # drop newly made column

Borough
BRONX            1617956
BROOKLYN         1739886
MANHATTAN        1055225
QUEENS            645971
STATEN ISLAND      87584
Unspecified       873221
Name: conditions_apply, dtype: int64
Borough
BROOKLYN    1739886
Name: conditions_apply, dtype: int64


Incident Zip
10001.0     9031
10002.0    32385
10003.0    25574
10004.0      329
10005.0      440
           ...  
11692.0    12498
11693.0     4768
11694.0    10557
11697.0      269
12345.0        1
Name: conditions_apply, Length: 202, dtype: int64

Incident Zip
11226.0    215709
Name: conditions_apply, dtype: int64
Incident Address
34 ARDEN STREET    14298
Name: conditions_apply, dtype: int64
Status
Assigned             4
Closed         5886253
In Progress        364
Open            133220
Pending              2
Name: conditions_apply, dtype: int64
Status
Closed    5886253
Name: conditions_apply, dtype: int64


### Module 3: Quiz on Affected Areas
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, which borough had the highest number of complaints submitted?
* For the complaint types that you selected in the previous module that had a total number that exceeded 800,000 complaints, which borough had the lowest number of complaints submitted?
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, which ZIP code had the highest number of complainted submitted?
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, the address 89-21 Elmhurst Avenue had the highest number of complainted submitted?
* For the complaint types that you selected in the previous module that had at least 800,000 complaints logged, how many of the submitted tickets were closed?

Based on the above groupby counts analysis, I had to guess some of the answers (particularly for the 1st one and for the last one because the correct answers seem far off than what my analysis revealed and I had to guess from the closest value or something like that. This time I was lucky, it all worked out for this particular quiz.

**Correct answers are:** (1) BRONX, (2) Staten Island, (3) 11226, (4) False, (5) 2,133,331 complaints.

## Module 4: What Is the Relationship between Housing Characteristics and Complaints?
Estimated time needed: 3 hours

### Objective for Exercise:
* Use your data visualization skills to visualize the data and extract meaningful patterns to guide the modelling process.

The goal of this exercise is to find the answer to the Question 3 of the problem statement:

Does the Complaint Type that you identified in response to Question 1 have an obvious relationship with any particular characteristic or characteristic of the Houses?

In this exercise, use the 311 dataset.

You also need to read back the PLUTO dataset from Cloud Object Store that you saved previously in the course. Use the PLUTO dataset for the borough that you already identified to focus on the last exercise.Ensure that you use only a limited number of fields from the dataset so that you are not consuming too much memory during your analysis.

The recommended fields are Address, BldgArea, BldgDepth, BuiltFAR, CommFAR, FacilFAR, Lot, LotArea, LotDepth, NumBldgs, NumFloors, OfficeArea, ResArea, ResidFAR, RetailArea, YearBuilt, YearAlter1, ZipCode, YCoord, and XCoord.

At the end of this exercise, you should determine whether the type of complaint that you have identified as the response to Question 1 has an obvious relationship with any particular characteristic or characteristics of the houses.



### Module 4: Quiz on Correlation
* Can you determine the age of some building from the PLUTO dataset? (0.0/5.0 points) - No, Yes, Not sure
* Which of the following methodologies can you use to identify whether any relationship exist between the building characteristics and the number of complaints? (0.0/7.0 points) - Pearson Correlation, Feature Importance using Decision Tree, None of the above
* The cardinality of the data in PLUTO dataset is at the same level as the complaint dataset? (0.0/7.0 points) - True or False


### Cardinality:
Ref: https://brilliant.org/wiki/cardinality/ 

The cardinality of a set is a measure of a set's size, meaning the number of elements in the set. For instance, the set A={1,2,4}A = \{1,2,4\} A={1,2,4} has a cardinality of 333 for the three elements that are in it. The cardinality of a set is denoted by vertical bars, like absolute value signs; for instance, for a set AAA its cardinality is denoted ∣A∣|A|∣A∣. When AAA is finite, ∣A∣|A|∣A∣ is simply the number of elements in AAA. When AAA is infinite, ∣A∣|A|∣A∣ is represented by a [cardinal number](https://brilliant.org/wiki/cardinality/#cardinal-numbers).

In [5]:
pathPL = "/Users/kpadhikari/Desktop/BigFls/DS_ML/PLUTO_for_WEB/"
filePL = "BK_18v1"
dfpl0 = pd.read_pickle('{}/{}.pkl'.format(pathPL,filePL))
display(dfpl0.columns)
dfpl0.head()

Index(['Borough', 'Block', 'Lot', 'CD', 'CT2010', 'CB2010', 'SchoolDist',
       'Council', 'ZipCode', 'FireComp', 'PolicePrct', 'HealthCenterDistrict',
       'HealthArea', 'SanitBoro', 'SanitDistrict', 'SanitSub', 'Address',
       'ZoneDist1', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'Overlay1',
       'Overlay2', 'SPDist1', 'SPDist2', 'SPDist3', 'LtdHeight', 'SplitZone',
       'BldgClass', 'LandUse', 'Easements', 'OwnerType', 'OwnerName',
       'LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea',
       'GarageArea', 'StrgeArea', 'FactryArea', 'OtherArea', 'AreaSource',
       'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront',
       'LotDepth', 'BldgFront', 'BldgDepth', 'Ext', 'ProxCode', 'IrrLotCode',
       'LotType', 'BsmtCode', 'AssessLand', 'AssessTot', 'ExemptLand',
       'ExemptTot', 'YearBuilt', 'YearAlter1', 'YearAlter2', 'HistDist',
       'Landmark', 'BuiltFAR', 'ResidFAR', 'CommFAR', 'FacilFAR', 'BoroCode',
       'BBL', 'CondoNo', 'Tra

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version
0,BK,1,1,302,21.0,NaN,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,3.000010e+09,11/26/2013,1,1.0,1.0,18V1
1,BK,1,50,302,21.0,2000.0,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,E-231,0.000000e+00,NaN,1,1.0,1.0,18V1
2,BK,1,7501,302,21.0,2000.0,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,3.000010e+09,03/04/2016,1,1.0,1.0,18V1
3,BK,3,1,302,21.0,3002.0,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1
4,BK,3,5,302,21.0,NaN,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,0.000000e+00,NaN,4,1.0,1.0,18V1


## Module 5: Predict Complaint Types
Estimated time needed: 3 hours

### Objective for Exercise:
* Use your machine learning skills to build a predictive model to help a business function more efficiently.
The goal of this exercise is to do Model Development and Validation to find the answer to the Question 4 of the problem statement:

Can a predictive model be built for future prediction of the possibility of complaints of the specific type that you identified in response to Question 1?

In this exercise, you will use a feature-engineered dataset to determine whether a predictive model can be built to predict the complaint (of the Complaint Type that you decided to focus on in Week 2) by using past data.

Using the best model, you need to predict the number of future complaints (of the Complaint Type that you decided to focus on in Question 1).



### Module 4: Predictive Model for the Top Complaint Type
* Can the model that you developed use Number of Floors in an address as a possible predictive feature? (0.0/7.0 points) - Yes, No
* Although you are developing a model for a particular Complaint Type, you can use data for other complaint types to build the model? (0.0/7.0 points) - Yes, No
* The features that you can use to build the model can come from which datasets? Select all that apply. (0.0/7.0 points) - Complaint dataset, PLUTO dataset, Any other dataset